In [1]:
import cv2 
import numpy as np

In [2]:
config_file='yolov3.cfg' 
weights_file= 'yolov3.weights' 
class_file='yolov3.txt' 

In [3]:
img_file = 'dog.jpg' 

In [4]:
def resizeframe(image, width=500):    
    w = float(width)    
    r = w / image.shape[1]    
    dim = (width, int(image.shape[0] * r))    
    resized = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)    
    xr = dim[0]/float(image.shape[1])    
    yr = dim[1]/float(image.shape[0])    
    return resized 

In [5]:
def get_output_layers(net):        
    layer_names = net.getLayerNames()        
    output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]    
    return output_layers 

In [6]:
def draw_prediction(img, class_id, confidence, x, y, x_plus_w, y_plus_h):    
    label = str(classes[class_id])    
    color = COLORS[class_id]    
    cv2.rectangle(img, (x,y), (x_plus_w,y_plus_h), color, 2)    
    cv2.putText(img, label, (x-10,y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2) 

In [7]:
scale = 0.00392 
classes = None 
with open(class_file, 'r') as f:    
    classes = [line.strip() for line in f.readlines()] 
COLORS = np.random.uniform(0, 255, size=(len(classes), 3)) 
net = cv2.dnn.readNet(weights_file, config_file) 

In [8]:
def yolov3_predict(image):    
    Width = image.shape[1]    
    Height = image.shape[0]    
    blob = cv2.dnn.blobFromImage(image, scale, (416,416), (0,0,0), True, crop=False)    
    net.setInput(blob)    
    outs = net.forward(get_output_layers(net)) 
    class_ids = []    
    confidences = []    
    boxes = []    
    conf_threshold = 0.5    
    nms_threshold = 0.4 
    for out in outs:        
        for detection in out:            
            scores = detection[5:]            
            class_id = np.argmax(scores)            
            confidence = scores[class_id]            
            if confidence > 0.5:                
                center_x = int(detection[0] * Width)                
                center_y = int(detection[1] * Height)                
                w = int(detection[2] * Width)                
                h = int(detection[3] * Height)                
                x = center_x - w / 2                
                y = center_y - h / 2                
                class_ids.append(class_id)                
                confidences.append(float(confidence))                
                boxes.append([x, y, w, h]) 
    indices = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold) 
    for i in indices:        
        i = i[0]        
        box = boxes[i]        
        x = box[0]        
        y = box[1]        
        w = box[2]        
        h = box[3]        
        draw_prediction(image, class_ids[i], confidences[i], round(x), round(y), round( x+w), round(y+h))

In [9]:
def yolov3_predict_file(img_file):    
    image = cv2.imread(img_file)    
    yolov3_predict(image)    
    cv2.imshow("object detection", image)    
    cv2.waitKey()    #cv2.imwrite("object-detection.jpg", image)    
    cv2.destroyAllWindows() 

In [11]:
cap1 = cv2.VideoCapture(0)    #(videofile) 
face_counter = 0 
flag = True 
frameno = 0 
frameinterval = 1  #10 
while cap1.isOpened():    
    ret1, frame1 = cap1.read()    
    frameno += 1    
    if not ((frameno % frameinterval) == 0):        
        continue    
    if ret1 :        
        frame1 = resizeframe(frame1)        
        frame = frame1.copy()        
        cv2.imshow('Frames- pause p; resume r; quit q',frame) 
        if cv2.waitKey(1) & 0xFF == ord('p'):   #pause            
            while (True):                
                if flag:                    
                    yolov3_predict(frame)                    
                    cv2.imshow('Frames- pause p; resume r; quit q',frame)                    
                    flag = False                
                if cv2.waitKey(1) & 0xFF == ord('r'):   #resume                    
                    flag = True                    
                    break 
        if cv2.waitKey(10) & 0xFF == ord('q'):   #quit            
            break    
    else:        
        break 
cap1.release() 
cv2.destroyAllWindows() 